# Tutorial 6: Galactic Binaries & RJMCMC

张铭昊 北京化工大学 zmh780674484@163.com

教程6：双星系统与随机跳跃马尔可夫链蒙特卡洛（RJMCMC）方法

In the sixth tutorial, we will examine Galactic Binary waveforms. We will then use them in fixed-dimensional MCMC and then in RJMCMC. We use RJMCMC to perform model selection on the number of sources in the data. 

  在教程6中，我们将研究双星系统波形，并在固定维度的MCMC（马尔可夫链蒙特卡洛）和RJMCMC（随机跳跃马尔可夫链蒙特卡洛）中使用这些波形。我们使用RJMCMC来对数据中的源数量进行模型选择。

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from lisatools.utils.constants import *
from copy import deepcopy  # can be useful
from lisatools.utils.constants import *
from lisatools.sensitivity import get_sensitivity

## Task 1: Build and plot a Galacic Binary waveform using `GBGPU`

We will start by generating Galactic Binary waveforms with `GBGPU`. Pick reasonable parameters, build a waveform and plot it against the LISA A channel TDI Sensitivity Curve (`A1TDISens`) in the characteristic strain representation. You can access the information after waveform generation as attributes on the class. This may be updated in the future.

Useful documentation:
* [GBGPU](https://mikekatz04.github.io/GBGPU/html/user/main.html#gbgpu.gbgpu.GBGPU)

任务1：使用GBGPU构建并绘制双星系统引力波波形
  使用GBGPU生成双星系统波形。根据给定的合理参数，生成一个波形，并将其与LISA A通道的TDI灵敏度曲线（A1TDISens）以特征应变表示进行绘制。在波形生成后，可以通过类的属性来访问相关信息。请注意，这些信息可能会在将来进行更新。

In [ ]:
# imports
from gbgpu.gbgpu import GBGPU

In [ ]:
gb = GBGPU()

In [ ]:
amp = 2e-23  
f0 = 3e-3  
fdot = 7.538331e-18  
fddot = 0.0 
phi0 = 0.1  
inc = 0.2  
psi = 0.3  
lam = 0.4  
beta = 0.5  

Tobs = 4 * YRSID_SI
    
gb.run_wave(amp, f0, fdot, fddot, phi0, inc, psi, lam, beta, N=256)  
plt.plot(gb.freqs[0], gb.freqs[0] * np.abs(gb.A[0]))
plt.plot(gb.freqs[0], get_sensitivity(gb.freqs[0], sens_fn="A1TDISens", return_type="char_strain"))


## Task 2: Run an MCMC over a single GB source

Run a fixed-dimensional MCMC run with a chosen GB source. Fix the sky location for now to simplify the problem computationally (this is especially important for the next section on RJ with GBs). So, you will sample over 6 of the 8 parameters. Discuss or think about reasonable priors for these parameters and how you would determine that. For simplicity, we recommend using tightly (but not too tightly) bound uniform distributions for this example setup.

There is a faster `get_ll` method on the `GBGPU` class. However, it may be easier to use the full `AnalysisContainer` setup. This will make the RJ part more straight forward, but is not actually ideal for fixed-dimensional MCMC on GBs. 

AFter the run is complete, plot the posterior distribution with `chainconsumer` or `corner`. 

任务2：建立运行一个关于单个双星系统引力波源的MCMC程序
  使用固定维度MCMC方法对选定的双星系统源进行运行。我们先固定天区位置。因此，你将对8个参数中的6个参数进行采样。思考或讨论这些参数的合理先验分布以及如何确定它们。为简单起见，我们建议在这个示例设置中使用紧密（但不要过于紧密）的均匀分布。

GBGPU类上有一个更快的get_ll方法。然而，使用完整的AnalysisContainer设置可能更容易。这将使RJ部分更加直观，但对于GBs的固定维度MCMC实际上并不理想。

在运行完成后，使用chainconsumer或corner绘制后验分布图。

In [ ]:
from eryn.prior import uniform_dist, ProbDistContainer
from lisatools.analysiscontainer import AnalysisContainer
from lisatools.datacontainer import DataResidualArray
from lisatools.sensitivity import AE1SensitivityMatrix
from eryn.ensemble import EnsembleSampler
from eryn.state import State

In [ ]:
priors = {"gb": ProbDistContainer({
    0: uniform_dist(1e-23, 1e-21),
    1: uniform_dist(0.00299995, 0.00300005),
    2: uniform_dist(1e-18, 1e-17),
    3: uniform_dist(0.0, 2 * np.pi),
    4: uniform_dist(0.0, np.pi),  
    5: uniform_dist(0.0, np.pi)
})}
lam = 4.13
beta = -0.15

# [lam, beta] of galactic center
default_values = np.array([lam, beta])

N_wave = 256
def like_wrap(x, analysis1, data, fd, Tobs, dt, default_values):
    input_parameters = np.zeros(9)
    input_parameters[np.array([0, 1, 2, 4, 5, 6])] = x
    input_parameters[np.array([7, 8])] = default_values
    
    gb.run_wave(*input_parameters, T=Tobs, dt=dt, N=N_wave)  # T=Tobs, oversample=4)
    
    A = gb.A[0]
    E = gb.E[0]
   

    start_ind = int(gb.freqs[0][0] / df)
    template = DataResidualArray(np.array([A, E]), f_arr=gb.freqs[0])
    data_tmp = DataResidualArray(data[:, start_ind:start_ind + N_wave], f_arr=fd[start_ind:start_ind + N_wave])
    sens_mat = AE1SensitivityMatrix(data_tmp.f_arr)
    analysis = AnalysisContainer(data_tmp, sens_mat)
    
    ll = analysis.template_likelihood(template)
    return ll

injection_params = np.array([
    amp, 
    f0,
    fdot,
    0.0,
    phi0,
    inc,
    psi,
    lam,
    beta
])
print(injection_params)

Tobs = YRSID_SI
dt = 5.0
N = int(Tobs / dt)
Tobs = N * dt
df = 1 / Tobs

f_arr = np.arange(0.0, 1/(2 * dt) + df, df)
data_orig = np.asarray(gb.inject_signal(*injection_params, T=Tobs, dt=dt, N=N_wave))
data = DataResidualArray(data_orig.copy(), f_arr=f_arr)
sens_mat = AE1SensitivityMatrix(f_arr)
analysis = AnalysisContainer(data, sens_mat)

sampling_injection_params = np.array([
    amp,
    f0,
    fdot,
    phi0,
    inc,
    psi
])
print(f0)
like_wrap(sampling_injection_params, analysis, data_orig, analysis.data_res_arr.f_arr, Tobs, dt, default_values)

## Task 3: RJ with GBs

Our final task will be to run RJMCMC on a few close Galactic Binaries. The key component here is the "global" Likelihood function. Work to build a function that takes from Eryn and adjustable length array of templates to be summed into a global template prior to the Likelihood computations. This will be a bit tedious, but is very important for understanding this process. 

There is another nuance in this problem that must be dealt with to get this all to work. In the fixed-dimensional case with one binary, the default stretch proposal is effectively invariant to the scale along each dimension as there is no mixing of dimensional information when making a  proposal, $\vec{Y} = \vec{X}_j + z\left(\vec{X}_i - \vec{X}_j\right)$. The default `GaussianMove` that we used in tutorial 5 requires an inversion of the covariance matrix. If we sample in the parameters we used above ($A$, $f_0$, $\dot{f}$, etc.), the scale differences between parameters will cause numerical issues with matrix inversion and multiplication. 

There is a small variety of ways to deal with this. Here are two possibilities:

1) You can log scale and reduce each parmeter so they are all of order 1. This would involve changing the priors and making sure you include this conversion in your Likelihood function. You can use `eryn.utils.TransformContainer` to do this conversion if you would like.
2) You can create your own proposal where you assume a diagonal covariance and generate the information yourself. In the answer key, I have chosen to take this route for this example. **Hint**: the Eryn tutorial has an example of this. 

If you really want to get fancy: 

In the setup described, every source that currently has `inds=True` will be moved together. That means, if a given walker has 5 sources, all 5 sources' parameters will change at the same time. This can hurt the acceptance fraction of these moves. In reality, you may want to use Gibbs sampling to sample one or a few sources at once. You can accomplish this using the `gibbs_sampling_setup` kwarg for `eryn.moves.Move`. 

Useful documentation:

* [MHMove](https://mikekatz04.github.io/Eryn/html/user/moves.html#eryn.moves.MHMove)
* [TransformContainer](https://mikekatz04.github.io/Eryn/html/user/utils.html#eryn.utils.TransformContainer)
* [Move](https://mikekatz04.github.io/Eryn/html/user/moves.html#eryn.moves.Move)

If you can run the sampler and confirm the Likelihoods are working, then consider this completed. The time alloted for the tutorial and the overall setup needed to run this RJ setup correclty require a lot more runtime for reasonable results. So, you can plot what comes out, but it will become more accurate as your run the sampler longer. 

任务3：随机跳跃与双星系统
  最后的任务是在几个接近的双星系统上运行RJMCMC。这里的关键组件是“全局”似然函数。需要构建一个函数，从Eryn和可调长度的模板数组中获取模板，并将其求和成一个全局模板，然后进行似然计算。这可能有点繁琐，但对于理解这个过程非常重要。

  在这个问题中还有一个细微之处，必须处理才能使其正常工作。在固定维度的情况下，对于一个二进制系统，使用默认的拉伸提案是对每个维度的尺度不变的，因为在方案形成时没有混合维度信息，即 Y =Xj+z(Xi−X j)。我们在第5个教程中使用的默认的GaussianMove需要求解协方差矩阵的逆。如果我们在上述参数中进行采样（A，f0，f˙等），则参数之间的尺度差异会导致矩阵求逆和乘法过程中的数值问题。

处理这个问题有几种方法。以下是两种可能性：

1）可以对参数进行对数缩放，并将它们缩小到数量级为1。这涉及改变先验分布，并确保在似然函数中包括此转换。如果需要，可以使用eryn.utils.TransformContainer来执行此转换。

2）可以创建自己的方案，假设一个对角协方差，并自己生成所需的信息。在答案中，我选择了这种方法作为示例。提示：Eryn教程中有一个示例。

如果你想更进一步：

  在所描述的设置中，当前具有inds=True的每个源都将同时移动。这意味着，如果一个行走者有5个源，那么这5个源的参数都会同时改变。这可能会影响这些移动的接受率。实际上，你可能希望使用Gibbs采样一次采样一个或几个源。可以使用eryn.moves.Move的gibbs_sampling_setup关键字参数来实现这一点
  
  
  
  
  在运行采样器并确认似然函数正常工作后，任务可以被视为完成。给定的教程时间和整体设置需要更长的运行时间才能获得合理的结果。因此，你可以绘制输出结果，但随着运行时间的增加，结果将变得更准确。

In [ ]:
# imports
from eryn.moves import MHMove

def fill_template(template, A, E, freqs, Tobs, dt):
    for i in range(A.shape[0]):
        start_ind = gb.start_inds[i]
        end_ind = start_ind + N_wave
        
        assert end_ind - start_ind == gb.freqs.shape[1]
        template[0, start_ind:end_ind] += A[i]
        template[1, start_ind:end_ind] += E[i]

def generate_global_template(template, params_all, Tobs, dt):
    gb.run_wave(*params_all.T, T=Tobs, dt=dt, N=N_wave)  # Te=Tobs, oversample=4)
    fill_template(template, gb.A, gb.E, gb.freqs, Tobs, dt)
    
def global_log_likelihood(params_all, analysis, Tobs, dt, default_values):
    input_parameters = np.zeros((params_all.shape[0], 9))
    input_parameters[:, np.array([0, 1, 2, 4, 5, 6])] = params_all
    input_parameters[:, np.array([7, 8])] = default_values
    
    
    template = np.zeros_like(analysis.data_res_arr[:])

    generate_global_template(template, input_parameters, Tobs, dt)
    template_in = DataResidualArray(template, f_arr=f_arr)
    ll = analysis.template_likelihood(template_in)
    return ll
injection_params_all = priors["gb"].rvs(size=5)
input_parameters = np.zeros((injection_params_all.shape[0], 9))
input_parameters[:, np.array([0, 1, 2, 4, 5, 6])] = injection_params_all
input_parameters[:, np.array([7, 8])] = default_values
data = np.zeros((2, len(f_arr)), dtype=complex)

generate_global_template(data, input_parameters, Tobs, dt)
inds = np.where(data[0])
plt.loglog(f_arr[inds], np.abs(data[0][inds]))

class NonScaledDiagonalGaussianMove(MHMove):
    def __init__(self, cov_all, *args, **kwargs):

        for branch_name, cov in cov_all.items():
            assert isinstance(cov, np.ndarray) and cov.ndim == 1

        self.cov = cov_all
        super(NonScaledDiagonalGaussianMove, self).__init__(*args, **kwargs)
        
    def get_proposal(self, branches_coords, random, branches_inds=None, **kwargs):

        assert branches_inds is not None
        new_points = {}
        for name, cov in self.cov.items():
            assert name in branches_coords
            coords = branches_coords[name]
            inds = branches_inds[name]
            
            ntemps, nwalkers, nleaves_max, ndim = coords.shape

           
            sigma = random.randn(ntemps, nwalkers)

            tmp = np.zeros_like(coords)

            tmp = coords + sigma[:, :, None, None] * cov[None, None, None, :]
            
            new_points[name] = np.zeros_like(coords)

            
            new_points[name][inds] = tmp[inds]

        
        factors = np.zeros((ntemps, nwalkers))
            
        return new_points, factors
    
    nleaves_max = {"gb": 10}
nleaves_min = {"gb": 0}
ndims = {"gb": 6}

nwalkers = 14
ntemps = 12

lam = 4.13
beta = -0.15

# [lam, beta] of galactic center
default_values = np.array([lam, beta])

cov    = {"gb": np.array([1e-24, 1e-10, 1e-19, 0.01, 0.01, 0.01])}
moves = NonScaledDiagonalGaussianMove(cov)

sampler_rj = EnsembleSampler(
    nwalkers,
    ndims,
    global_log_likelihood,
    priors,
    branch_names=["gb"],
    nleaves_max=nleaves_max,
    nleaves_min=nleaves_min,
    args=(analysis, Tobs, dt, default_values),
    rj_moves=True,
    moves=moves,
    tempering_kwargs=dict(ntemps=ntemps)
)

start_points = {"gb": priors["gb"].rvs(size=(ntemps, nwalkers, nleaves_max["gb"]))}
inds_tmp = np.zeros((ntemps, nwalkers, nleaves_max["gb"]), dtype=bool)
inds_tmp[:, :, 1] = True

start_state = State(start_points, inds={"gb": inds_tmp})


nsteps = 10
end_state = sampler_rj.run_mcmc(start_state, nsteps, progress=True)

总结与收获
  第六讲教程把之前学习的方法应用到双星系统上，在任务1中我们定义参数的先验分布、参数和变量，同时定义似然函数接受一组参数，并基于这些参数计算模型的输出波形。然后，使用模型输出的波形和实际观测数据计算似然函数的值，最终获得似然函数。
  在任务2中执行基于MCMC的参数空间采样，生成全局模板，然后使用自定义的移动类在参数空间中生成方案点，使用EnsembleSampler执行MCMC采样，伴随其后输出最终的采样状态。
  经历了6讲的学习，从一开始的一些基本数值定义与计算到各种数据分析方法在各个天体系统的复杂应用都让我大开眼界，但仅开眼界是不够的，我深知我对这些模型背后物理图像的理解和编程计算能力都有所欠缺，因此我希望通过这次科创计划我能够进一步学习引力波数据处理的知识，并提高自己的编程水平。